# Traffic Analysis
This notebook will serve as the primary means of documentation for creating a traffic analysis model

_______________________________________________________________________________________________________________________________________________

### Full group includes:
- Jordan Brown
- Dylan Roy
- Maxwell Barret
- Julia Dewhurt

### Primary notebook contributers:

***The primary notebook contributers are the group members who were assigned to this specific task. All group members will work in collaboration to create a final working product. However, the nature of this project calls for the full group to be assigned to primary tasks.***

- ""
- ""
_______________________________________________________________________________________________________________________________________________

## Notebook Goal/Purpose

This notebook will be used to develop and test a model for analyzing and predicting the flow of traffic. This will involve data visualization, feature selection, and baseline comparison. The final intent for the dataloader is that this method will be able to take information from our object detection dataset and use that train local traffic systems on an ever increasing dataset. This will hopefully allow for improved traffic flow and a better understanding of the area for future city planning.
_______________________________________________________________________________________________________________________________________________

## Code Cells should use the following layout template

```
"""
At the top will be a commment block
this comment block will describe the
primary purpose for the cells code.
"""

# A comment stating used libary(s)

Code
    Implementation
        Goes
            Here

```

In [ ]:
"""
This cell will contain the import statements for the notebook.
"""